In [1]:
import requests
url = "https://www.naukri.com/data-science-jobs"
resp=requests.get(url)

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
import os
os.chdir("../chrome_driver")
from selenium import webdriver
browser2 = webdriver.Chrome()

In [6]:
browser2.get(url)

In [7]:
raw_html=browser2.page_source

In [8]:
parsed_html=BeautifulSoup(raw_html,"html.parser")

In [9]:
Naukri_dsjobs = pd.DataFrame(columns=("Job_id","Company","Experience","Salary","Location","Job_link","Skillset","Posted"))

In [10]:
def get_data(parsed_html):
    table = pd.DataFrame()
    job_id = []
    company_name=[]
    Exp = []
    Salary = []
    Location = []
    Link = []
    Skillset = []
    Posted = []
    for row in parsed_html.find_all("article",class_="jobTuple bgWhite br4 mb-8"):
        job_id.append(row["data-job-id"])
        company_name.append(row.find("div",class_="mt-7 companyInfo subheading lh16").a.get_text())
        Link.append(row.find("div",class_="mt-7 companyInfo subheading lh16").a['href'])
        try:
            Exp.append(row.find("li",class_="fleft grey-text br2 placeHolderLi experience").get_text())
        except:
            Exp.append("No Data")
        try:
            Salary.append(row.find("li",class_="fleft grey-text br2 placeHolderLi salary").get_text())
        except:
            Salary.append("No data")
        try:
            Location.append(row.find("li",class_="fleft grey-text br2 placeHolderLi location").get_text())
        except:
            Location.append("No Data")
        try:
            skills = row.find_all("li",class_="fleft fs12 grey-text lh16 dot")
            Skillset.append((list(map(lambda x: x.text,skills))))
        except:
            Skillset.append("No Data")
        try:
            Posted.append(row.find("div",class_="jobTupleFooter mt-20").span.get_text())
        except:
            Posted.append("No Data")

        table = pd.DataFrame({"Job_id":job_id,"Company":company_name,"Experience":Exp,"Salary":Salary,"Location":Location,"Job_link":Link,"Skillset":Skillset,"Posted":Posted})
    return table

In [12]:
import time
from tqdm import tqdm
base_url = "https://www.naukri.com/data-science-jobs-{}"
for page_num in tqdm(range(1,401)):
    url=base_url.format(page_num)
    browser1 = webdriver.Chrome()
    browser1.get(url)
    time.sleep(1)
    raw_data=browser1.page_source
    browser1.close()
    html=BeautifulSoup(raw_data,"html.parser")
    Naukri_dsjobs = Naukri_dsjobs.append(get_data(html),ignore_index = True)


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [5:14:59<00:00, 18.90s/it]


In [13]:
Naukri_dsjobs.shape

(13357, 8)

In [14]:
Naukri_dsjobs.to_csv('naukri.csv')

In [15]:
os.getcwd()

'E:\\soumya\\herovired\\webscrapngpython\\chrome_driver'